<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/colab_cucat_bench_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cu_cat CPU, GPU benchmark

This notebook examines `cu-cat` automatic feature engineering performance on variously populated datasets, ranging in size and complexity of data. The `dirty_cat` engine is highly optimized and parallelized for CPUs, and the `cu-cat` engine further adds single-GPU acceleration.
* Advantage can be seen at small scales, but GPU extraction really shines with scale, as GPU memory is loaded close to but not exceeding vram.
*The benchmark does not examine bigger-than-memory and distributed scenarios.
*There is a tradeoff between compute speedup vs gpu mem swapping time, and some cases are not to the advantage of the GPU.
Both the `GapEncoder()` and `TableVectorizer()` methods are employed independently here, as well as more extensively within the framework of `graphistry`'s `.featurize()` and `.umap()` functions.

The provided results here are from running on a free Google Colab T4 runtime, with a 2.2GHz Intel CPU (12 GB CPU RAM) and T4 Nvidia GPU (16 GB GPU RAM).

In [ ]:
# !pip install --extra-index-url=https://pypi.nvidia.com cuml-cu12==23.12.00 cudf-cu12==23.12.00 #==23.12.00 #cugraph-cu11 pylibraft_cu11 raft_dask_cu11 dask_cudf_cu11 pylibcugraph_cu11 pylibraft_cu11
# !pip install -U git+https://github.com/graphistry/pygraphistry.git@75ac2b059c95f833ed61fe2062a43afba1f0b979 # dev/depman_gpufeat
# !pip3 install cu_cat

In [1]:
import cuml
cuml.__version__

'23.12.00'

In [2]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings,time
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)
import cudf,cuml,cupy

In [3]:
import cu_cat
print('cu_cat version:',cu_cat.__version__)
import sklearn
print('sklearn version:',sklearn.__version__,'is <1.3.2')

cu_cat version: v0.9.11
sklearn version: 1.2.2 is <1.3.2


In [4]:
import graphistry
# graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='***', password='***') ## key id, secret key
# graphistry.__version__

In [5]:
# askHNA = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

# askHN = askHNA.sample(50,replace=False)
# g = graphistry.nodes((askHN))
# pr = cProfile.Profile()
# pr.enable()
# g5b = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat',cardinality_threshold=10,cardinality_threshold_target=10)
# pr.disable()
# stats = Stats(pr)
# stats.sort_stats('tottime').print_stats(20)

In [6]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.ERROR, force=True, format='%(asctime)s %(levelname)s %(message)s', datefmt='%H:%M:%S')

# logging.basicConfig(filename='app.log',
                    # level=logging.DEBUG,
                    # force=True, # Resets any previous configuration
                    # )

In [ ]:
# import cupy as cp
# cp._default_memory_pool.free_all_blocks()
# import gc
# gc.collect()

In [ ]:
# !git clone https://github.com/graphistry/pygraphistry.git
# !pip install sentence-transformers umap-learn parameterized dirty_cat
# # !cd pygraphistry
# # !git checkout dev/depman_gpufeat

# !echo PYTHONPATH="${PYTHONPATH}:/path/to/directory" >> .env
# !echo PYTHONPATH="${PYTHONPATH}:graphistry/tests" >> .env
# !echo PYTHONPATH="${PYTHONPATH}:graphistry" >> .env
# # export PYTHONPATH="${PYTHONPATH}:graphistry"
# # from (del graphistry.)tests.test_feature_utils import
# # is_test_cudf = cudf # and os.environ["TEST_CUDF"] != "0"
# # !git clone https://github.com/graphistry/cu-cat.git

# # !pytest graphistry/tests/test_feature_utils.py
# # !pytest graphistry/tests/test_umap_utils.py
# # !pytest graphistry/tests/test_compute.py::TestComputeMixin::test_keep_nodes_cudf

## Introducing `cu-cat`
### with GPU-accelerated `GapEncoder` and `TableVectorizer` (prev. `SuperVectorizer`)

In [ ]:
import dirty_cat
dirty_cat.__file__

'/usr/local/lib/python3.10/dist-packages/dirty_cat/__init__.py'

In [ ]:
from dirty_cat.datasets._fetching import fetch_midwest_survey
from sklearn.model_selection import train_test_split
from cu_cat import GapEncoder as cuGapEncoder, TableVectorizer as cuTable
from dirty_cat import GapEncoder as dGapEncoder, TableVectorizer as dTable
from numpy.testing import assert_array_equal

In [ ]:
dataset = fetch_midwest_survey()
X_train, X_test = train_test_split(
    dataset.X[["What_would_you_call_the_part_of_the_country_you_live_in_now"]],
    random_state=0,
)
encA = cuGapEncoder(n_components=2, random_state=2)
encA.fit_transform(cudf.from_pandas(X_train))
topics1 = encA.get_feature_names_out()

encB = dGapEncoder(n_components=2, random_state=2)
encB.fit_transform(X_train)
topics2 = encB.get_feature_names_out()

assert len(topics1) == len(topics2)


In [ ]:
cc_table_vec = cuTable()
aa = cc_table_vec.fit_transform((dataset.X))
C_out = cc_table_vec.transformers_
print(C_out[2]) # print high_card_cat
d_table_vec = dTable()
aa = d_table_vec.fit_transform((dataset.X))
D_out = d_table_vec.transformers_
print(D_out[1]) # print high_card_cat

## Datasets Explored in the notebook:

we

| Dataset | Rows     | Columns     | Bench dim(s)   | Descriptions |
|-------------|-----------|-----------|---|---|
| [**REDDIT**](#ndf_reddit)| 100     | 6     |  |text-rich|
| [**CTU-13**](#ctu-13) | 10k    | 16  |  |IP-address, datetime, numeric, short text labels|
| [**redteam**](#red)   | 20k   | 14  | |messy string, text, other|
| [**ask HN**](#hn)   | 3000 | 14 | |title, text, datetime,numerics|
| [**20newsgroups**](#20n)   | 11k   | 1  | |paragraphs|
| [**winlogs**](#win)   | 5M | 21 | |windows log data: sparse,incomplete, hectic|


<a name="ndf_reddit"></a>
##ndf_reddit

100 rows of `title` and `document` text-rich columns

In [ ]:
import pandas as pd
import graphistry
from graphistry.features import topic_model

ndf_reddit = pd.read_csv('https://raw.githubusercontent.com/graphistry/pygraphistry/master/graphistry/tests/data/reddit.csv')#'pygraphistry/graphistry/tests/data/reddit.csv')
print(ndf_reddit.shape)
ndf_reddit.head(5)

(100, 6)


,Unnamed: 0,title,document,user,type,label
0,856368,Mother has short term memory issues at her early 40’s,She tells me a joke or news about work and she’ll tell me again 10h later and then the next day. She has to tell me something 3 times under 2 days and then she remembers she told me it. It isn’t f...,None,subs,neuro
1,929759,Could I buy a company with stocks?,"I'm an amateur investor. Meaning I know nothing about stocks or how they work. I'm looking to get a crash course in some angles. So let's say I have a company that I really like, but I really want...",None,subs,investing
2,1163407,on He Listed a T. Rex Fossil on eBay for $2.95 Million. Scientists Weren’t Thrilled.,"Sure, we should pay in proportion to the amount of the genetic code of the T. Rex invented by the fossil hunter. Everyone should be compensated for the things that they create. What's that, all he...",None,comments,biology
3,2032529,"on Stripper in Oregon, not sure how to estimate annual income for quarterly payments, ALSO do I have to pay a state and federal SE tax??","It's going to depend on what your overall income is for the year, from all sources of income. Your dancing income is self-employment income, and on a Schedule C (or C-EZ) you will list your income...",None,comments,tax
4,6728650,which software should i get for cold calls?,So I hired couple of people who are going to do cold calls for me 5-6 days a week 8 hours a day and I am way to busy to baby sit them. I was wondering what software I should use to monitor their p...,/u/polo1990,summary,realtors


In [ ]:
g = graphistry.nodes(ndf_reddit)
print(g._nodes.shape)

(100, 6)


In [ ]:
## run featurize via CPU and GPU and compare speeds, with both results being UMAP-ed on GPU
prA = cProfile.Profile()
prA.enable()
g = graphistry.nodes(ndf_reddit)
start0 = time.time()
g0a = g.umap(**topic_model,engine='cuml',memoize=False,feature_engine='dirty_cat')#,cardinality_threshold=10,cardinality_threshold_target=10)
end0 = time.time()
prA.disable()
with open('reddit_dirty_.txt', 'w') as stream:
    stats = Stats(prA,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

g = graphistry.nodes(cudf.from_pandas(ndf_reddit))
prB = cProfile.Profile()
prB.enable()
start1 = time.time()
g0b = g.umap(**topic_model,engine='cuml',memoize=False,feature_engine='cu_cat')#,cardinality_threshold=10,cardinality_threshold_target=10)
end1 = time.time()
prB.disable()
with open('reddit_cucat_.txt', 'w') as stream:
    stats = Stats(prB,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

del g, ndf_reddit
T0 = end0-start0
T1 = end1-start1
print('\nFull reddit dataset: \nCPU dirty_cat runtime:',np.round(T0,4),'\nGPU cu_cat runtime:',np.round(T1,4),'\nspeedup:', np.round(T0/T1,4))



Full reddit dataset: 
CPU dirty_cat runtime: 17.6901 
GPU cu_cat runtime: 8.3635 
speedup: 2.1152


### create an account at graphistry.com to plot output

In [ ]:
# g0a.plot()

In [ ]:
# g0b.plot()

In [ ]:
del g0a, g0b

<a name="ctu-13"></a>
## CTU-13 malware dataset
roughly 10k rows of IP-address, date-time, numeric and short **text** labels

In [ ]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7

Botnet abundance: 0.69%


In [ ]:
g = graphistry.edges(edf, src, dst).materialize_nodes()
print(g._nodes.shape, g._edges.shape)
g._nodes.head(5)

(4155, 1) (9911, 16)


,id
0,147.32.84.165
1,147.32.84.138
2,147.32.84.134
3,147.32.86.165
4,147.32.84.21


In [ ]:
## run featurize via CPU and GPU and compare speeds, with both results being UMAP-ed on GPU
prA = cProfile.Profile()
prA.enable()
g = graphistry.edges(edf, src, dst)
# g = graphistry.nodes(edf[['SrcAddr','DstAddr']])
start0 = time.time()
g1a = g.umap(kind='edges',
            X=good_cols_with_edges,
            y = ['bot'],
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2,
            n_topics=n_topics,
            feature_engine='dirty_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean',
            n_neighbors=12)
end0 = time.time()
prA.disable()
with open('ctu13_dirty_.txt', 'w') as stream:
    stats = Stats(prA,stream=stream)
    stats.sort_stats('tottime').print_stats(20)
del g

g = graphistry.edges(cudf.from_pandas(edf), src, dst)
# g = graphistry.nodes(edf[['SrcAddr','DstAddr']])
prB = cProfile.Profile()
prB.enable()
start1 = time.time()
g1b = g.umap(kind='edges',
            X=good_cols_with_edges,
            y = ['bot'],
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2,
            n_topics=n_topics,
            feature_engine='cu_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean',
            n_neighbors=12)
end1 = time.time()
prB.disable()
with open('ctu13_cucat_.txt', 'w') as stream:
    stats = Stats(prB,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

del g, edf
T0 = end0-start0
T1 = end1-start1
print('\nFull CTU dataset: \nCPU dirty_cat runtime:',np.round(T0,4),'\nGPU cu_cat runtime:',np.round(T1,4),'\nspeedup:', np.round(T0/T1,4))



Full CTU dataset: 
CPU dirty_cat runtime: 46.1048 
GPU cu_cat runtime: 8.7238 
speedup: 5.285


In [ ]:
# g1a.plot()

In [ ]:
# g1b.plot()

In [ ]:
del g1a,g1b

<a name="red"></a>
## redteam

around 20k rows of messy string, text and other columns needing

In [8]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))

In [9]:
tdf.head(5)

,index,time,src_domain,src_computer,dst_computer,feats,RED,dst_domain,auth_type,logontype,authentication_orientation,success_or_failure,feats2,node
0,0,150885,U620@DOM1,C17693,C1003,C17693 C1003,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,151036,U748@DOM1,C17693,C305,C17693 C305,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2,151648,U748@DOM1,C17693,C728,C17693 C728,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2
3,3,151993,U6115@DOM1,C17693,C1173,C17693 C1173,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3
4,4,153792,U636@DOM1,C17693,C294,C17693 C294,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4


In [ ]:
## run featurize via CPU and GPU and compare speeds, with both results being UMAP-ed on GPU
for n_samples in [5000,10000,15000]:
    tdfA=tdf.sample(n_samples,replace=False)
    pr = cProfile.Profile()
    pr.enable()
    g = graphistry.nodes((tdfA), 'node')
    start0 = time.time()
    g2a = g.umap(X=['feats'],
                min_words=1000000, # force high so that we don't use Sentence Transformers
                cardinality_threshold=4, # set low so we force topic model
                n_topics=32, # number of topics
                use_scaler=None,
                feature_engine='dirty_cat',
                memoize=False,
                engine='cuml',
                use_scaler_target=None
            )
    end0 = time.time()
    pr.disable()
    with open('red_dirty_',n_samples,'.txt', 'w') as stream:
        stats = Stats(pr,stream=stream)
        stats.sort_stats('tottime').print_stats(20)
    del g
    g = graphistry.nodes(cudf.from_pandas(tdfA), 'node')
    pr = cProfile.Profile()
    pr.enable()
    start1 = time.time()
    g2a = g.umap(X=['feats'],
                min_words=1000000, # force high so that we don't use Sentence Transformers
                cardinality_threshold=4, # set low so we force topic model
                n_topics=32, # number of topics
                use_scaler=None,
                feature_engine='cu_cat',
                memoize=False,
                engine='cuml',
                use_scaler_target=None
            )
    end1 = time.time()
    pr.disable()
    with open('red_cucat_',n_samples,'.txt', 'w') as stream:
        stats = Stats(pr,stream=stream)
        stats.sort_stats('tottime').print_stats(20)

    del g, tdfA
    T0 = end0-start0
    T1 = end1-start1
    print('\nn_samples of redteam data:',n_samples,'\nCPU dirty_cat runtime:',np.round(T0,4),'\nGPU cu_cat runtime:',np.round(T1,4),'\nspeedup:', np.round(T0/T1,4))


In [ ]:
# g2a.plot()

In [ ]:
# g2b.plot()

<a name="hn"></a>
## ask HN
3000 rows of `title` and `text` columns similar to ndf_reddit dataframe, plus several disparate `time-stamp` columns and various numerics

In [12]:
# # get the data top 3000 posts on Hacker News
askHNA = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)
print(askHNA.shape)
askHNA.head(5)

(3000, 14)


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,"I'm a software engineer going blind, how should I prepare?",NaN,"I&#x27;m a 24 y&#x2F;o full stack engineer (I know some of you are rolling your eyes right now, just highlighting that I have experience on frontend apps as well as backend architecture). I&#x27;v...",NaN,zachrip,3270,1587332026,2020-04-19 21:33:46+00:00,story,22918980,NaN,473.0,NaN,NaN
1,Am I the longest-serving programmer – 57 years and counting?,NaN,"In May of 1963, I started my first full-time job as a computer programmer for Mitchell Engineering Company, a supplier of steel buildings. At Mitchell, I developed programs in Fortran II on an IB...",NaN,genedangelo,2634,1590890024,2020-05-31 01:53:44+00:00,story,23366546,NaN,531.0,NaN,NaN
2,Is S3 down?,NaN,I&#x27;m getting<p>{\n &quot;errorCode&quot; : &quot;InternalError&quot;\n}<p>When I attempt to use the AWS Console to view s3,NaN,iamdeedubs,2589,1488303958,2017-02-28 17:45:58+00:00,story,13755673,NaN,1055.0,NaN,NaN
3,What tech job would let me get away with the least real work possible?,NaN,"Hey HN,<p>I&#x27;ll probably get a lot of flak for this. Sorry.<p>I&#x27;m an average developer looking for ways to work as little as humanely possible.<p>The pandemic made me realize that I do no...",NaN,lmueongoqx,2022,1617784863,2021-04-07 08:41:03+00:00,story,26721951,NaN,1091.0,NaN,NaN
4,What books changed the way you think about almost everything?,NaN,I was reflecting today about how often I think about Freakonomics. I don&#x27;t study it religiously. I read it one time more than 10 years ago. I can only remember maybe a single specific anecdot...,NaN,anderspitman,2009,1549387905,2019-02-05 17:31:45+00:00,story,19087418,NaN,1165.0,NaN,NaN


In [ ]:
## run featurize via CPU and GPU and compare speeds, with both results being UMAP-ed on GPU
for n_samples in [500,1000,2000]:
    askHN = askHNA.sample(n_samples,replace=False)
    pr = cProfile.Profile()
    pr.enable()
    g = graphistry.nodes(askHN)
    start0 = time.time()
    g5a = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat',cardinality_threshold=10,cardinality_threshold_target=10)
    end0 = time.time()
    pr.disable()
    with open('hn_dirty_'+str(n_samples)+'.txt', 'w') as stream:
        stats = Stats(pr,stream=stream)
        stats.sort_stats('tottime').print_stats(20)
    del g
    g = graphistry.nodes((askHN))
    pr = cProfile.Profile()
    pr.enable()
    start1 = time.time()
    g5b = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat',cardinality_threshold=10,cardinality_threshold_target=10)
    end1 = time.time()
    pr.disable()
    with open('hn_cucat_'+str(n_samples)+'.txt', 'w') as stream:
        stats = Stats(pr,stream=stream)
        stats.sort_stats('tottime').print_stats(20)

    del g, askHN
    T0 = end0-start0
    T1 = end1-start1
    print('\nn_samples of HN data:',n_samples,'\nCPU dirty_cat runtime:',np.round(T0,4),'\nGPU cu_cat runtime:',np.round(T1,4),'\nspeedup:', np.round(T0/T1,4))


In [ ]:
from graphistry.dep_manager import deps
[deps.cudf, deps.cuml, deps.cu_cat]

In [ ]:
# g3a.plot()

In [ ]:
# g3b.plot()

<a name="20n"></a>
## 20newsgroups
11k massive, single column containing multi-sentences to multi-paragraphs

In [14]:
from sklearn.datasets import fetch_20newsgroups
news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)
# print(news.shape)
# news.head(5)

In [15]:
# g = graphistry.nodes(news)
# print(g._nodes.shape, g._edges.shape)
# g._nodes.head(5)

In [ ]:
## run featurize via CPU and GPU and compare speeds, with both results being UMAP-ed on GPU
for n_samples in [500,1000,2000]:
    newsA = news[:n_samples]
    newsA=pd.DataFrame(newsA)
    pr = cProfile.Profile()
    pr.enable()
    g = graphistry.nodes(newsA)
    start0 = time.time()
    g5a = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat',cardinality_threshold=10,cardinality_threshold_target=10)
    end0 = time.time()
    pr.disable()
    with open('news_dirty_'+str(n_samples)+'.txt', 'w') as stream:
        stats = Stats(pr,stream=stream)
        stats.sort_stats('tottime').print_stats(20)

    g = graphistry.nodes(cudf.from_pandas(newsA))
    pr = cProfile.Profile()
    pr.enable()
    start1 = time.time()
    g5b = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat',cardinality_threshold=10,cardinality_threshold_target=10)
    end1 = time.time()
    pr.disable()
    with open('news_cucat_'+str(n_samples)+'.txt', 'w') as stream:
        stats = Stats(pr,stream=stream)
        stats.sort_stats('tottime').print_stats(20)

    del g, newsA
    T0 = end0-start0
    T1 = end1-start1
    print('\nn_samples of news data:',n_samples,'\nCPU dirty_cat runtime:',np.round(T0,4),'\nGPU cu_cat runtime:',np.round(T1,4),'\nspeedup:', np.round(T0/T1,4))


In [ ]:
# g4a.plot()

In [ ]:
# g4b.plot()

<a name="win"></a>
## winlogs
5M rows of windows log data, including 21 columns of sparse, incomplete and generally hecticly "structured" data. `cu-cat` shines as you scale rows up!

In [ ]:
!wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogs=pd.read_parquet('part.88.parquet')
print(winlogs.shape)
winlogs.head(5)

In [ ]:
g = graphistry.nodes(winlogsA)
# print(g._nodes.shape, g._edges.shape)
g._nodes.head(5)

In [ ]:

for n_samples in [5000,10000,20000]:
    winlogsA=winlogs.sample(n_samples,replace=False)
    pr = cProfile.Profile()
    pr.enable()
    g = graphistry.nodes(winlogsA)
    start0 = time.time()
    g5a = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat',cardinality_threshold=10,cardinality_threshold_target=10)
    end0 = time.time()
    pr.disable()
    with open('winlogs_dirty_'+n_samples+'.txt', 'w') as stream:
        stats = Stats(pr,stream=stream)
        stats.sort_stats('tottime').print_stats(20)

    g = graphistry.nodes(cudf.from_pandas(winlogs))
    pr = cProfile.Profile()
    pr.enable()
    start1 = time.time()
    g5b = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat',cardinality_threshold=10,cardinality_threshold_target=10)
    end1 = time.time()
    pr.disable()
    with open('winlogs_cucat_'+n_samples+'.txt', 'w') as stream:
        stats = Stats(pr,stream=stream)
        stats.sort_stats('tottime').print_stats(20)

    del g, winlogsA
    T0 = end0-start0
    T1 = end1-start1
    print('\nn_samples of winlogs data:',n_samples,'\nCPU dirty_cat runtime:',np.round(T0,4),'\nGPU cu_cat runtime:',np.round(T1,4),'\nspeedup:', np.round(T0/T1,4))


In [ ]:
# g5a.plot()

In [ ]:
# g5b.plot()